# use Beart-uncased model from HuggingFace
* https://huggingface.co/google-bert/bert-base-uncased

In [ ]:
# !pip install transformers

# download pre-trained BERT model

In [ ]:
from transformers import BertModel, BertTokenizer

In [ ]:
model = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# to encode Sentences

In [ ]:
sample_text = "Virat Kohli is a top run scorrer batter in the world from india."

In [ ]:
input_id = tokenizer.encode(sample_text, return_tensors="pt")
input_id # [SOS]==101, [EOS]==102

tensor([[  101,  6819,  8609, 12849, 27766,  2003,  1037,  2327,  2448,  8040,
          2953, 14544, 23801,  1999,  1996,  2088,  2013,  2634,  1012,   102]])

In [ ]:
import torch

with torch.no_grad():
    embending = model(input_id)

In [ ]:
embending.last_hidden_state

tensor([[[-0.6368, -0.2556, -0.1871,  ...,  0.0759,  0.6348,  0.0476],
         [ 1.2011, -1.6011, -0.7387,  ...,  0.5350,  0.5177,  0.0169],
         [ 0.0655, -0.9399, -0.3858,  ...,  0.5138, -0.3939, -1.8243],
         ...,
         [-0.2498, -0.6050, -0.6025,  ..., -0.1965,  0.0610, -0.1747],
         [ 0.5948, -0.0471, -0.2915,  ...,  0.1310, -0.4105, -0.4018],
         [ 0.6200,  0.1058, -0.2052,  ...,  0.0485, -0.4901, -0.3521]]])

In [ ]:
embending.last_hidden_state.mean(dim=1).shape

torch.Size([1, 768])

# `Spam Classifier using BERT encoding`

In [ ]:
import pandas as pd
df = pd.read_csv("spam.csv", encoding="ISO-8859-1")
df = df.rename(columns={"v2":"email", "v1":"label"})[["email","label"]]
df.head()

,email,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [ ]:
import re
def cleanEmail(str_):
    str_ = str_.lower()
    str_ = re.sub("[^a-z0-9]+", " ", str_)
    return str_

df["clean_email"] = df["email"].apply(cleanEmail)
df.head()

,email,label,clean_email
0,"Go until jurong point, crazy.. Available only ...",ham,go until jurong point crazy available only in ...
1,Ok lar... Joking wif u oni...,ham,ok lar joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,free entry in 2 a wkly comp to win fa cup fina...
3,U dun say so early hor... U c already then say...,ham,u dun say so early hor u c already then say
4,"Nah I don't think he goes to usf, he lives aro...",ham,nah i don t think he goes to usf he lives arou...


In [ ]:
sentences = df["clean_email"].tolist()

In [ ]:
texts = [text for text in sentences if text.strip()] # looking for blank sentences
len(sentences), len(texts) # 2 blank sentence

(5572, 5570)

In [ ]:
# encoding for sentences
import torch

def get_encoding(sentences):
    inputs = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs) # tokenizer.encode == **inputs

    embendings = outputs.last_hidden_state.mean(dim=1)
    return embendings

In [ ]:
x = get_encoding(sentences[:500])
y = df["label"].iloc[:500]

In [ ]:
# x is tensor so convert tensor to ndarray
x1 = x.cpu().detach().numpy() # require when using bert embending within classical ML approach

In [ ]:
type(x), type(x1)

(torch.Tensor, numpy.ndarray)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x1, y, test_size=0.3, random_state=42)

In [ ]:
# ML Model
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()

In [ ]:
rf_model.fit(x_train, y_train)

RandomForestClassifier()

In [ ]:
y_pred = rf_model.predict(x_test)

In [ ]:
from sklearn.metrics import *

In [ ]:
accuracy_score(y_test, y_pred)*100

99.33333333333333

# `Quesion Answering Model`:

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

In [ ]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokeniser = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
para = "Virat Kohli is married to the famous Bollywood actress Anushka Sharma, who is one of the biggest stars of the Hindi film industry\
          Virat and Anushka started dated in 2013, after which the media gave them the name Virushka.\
          The couple kept their relationship really private until they suddenly got married on 11 December 2017, in a beautiful private function in Florence, Italy.\
          Attended by only very close family members and friends, their wedding photos and happy relationships set new goals for couple goals in India.\
          Virat has cited that Anushka made him more emotionally strong and mature. They are together and rank among the most popular and followed couples in the country.\
          Virat and Anushka have turned parents to their first child, Vamika Kohli, born on January 11, 2021. Kohli has mentioned that Vamika, his daughter's name, is another name for Goddess Durga.\
          The couple welcomed their second baby, Akaay Kohli, on February 15, 2024.\
          His National Honours: 2013 – Arjuna Award, second highest sporting honour. 2017 – Padma Shri, India's fourth highest civilian award. 2018 – Major Dhyan Chand Khel Ratna Award, India's highest sporting honour.\
          Test records: Highest Number of Wins as Captain of India: 40 wins in 68 matches, Four Test double hundreds in four successive series\
          ODI records: scored the most ODI centuries (50), He has the record of most ODI centuries while chasing (27), Virat has most number of ODI centuries in India (22), Fastest to - 8,000 runs (175 innings), 9,000 runs (194 innings), 10,000 runs (205 innings), 11,000 runs (222 innings), 12,000 runs (242 innings), and 13,000 runs (267 innings)\
          T20I records: He scored the most runs in T20I (4,042 runs) Highest career batting average in T20I – 51.75. Fastest to reach - 3,000 runs (81 innings), 3,500 runs (96 innings), and 4,000 runs (107 innings) Virat got the most Player of the Match awards (15 times) and Player of the Series awards (7 times)\
          IPL records: Virat is the leading run-scorer in Indian Premier League (8,004) Most runs in a single edition of IPL (973 runs in 2016). Only player to be involved in three double-century plus stands, two times with AB de Villiers and once with Chris Gayle. Most hundreds in the league (8) and in a single season (4)."

In [ ]:
que = "where's virat mareied?"

In [ ]:
input = tokeniser(que, para, return_tensors='pt', truncation=True)

In [ ]:
with torch.no_grad():
    output = model(**input)

In [ ]:
# output # start_logits, end_logits

In [ ]:
start_logits = output.start_logits
end_logits = output.end_logits

In [ ]:
start_idx = torch.argmax(start_logits)
end_idx = torch.argmax(end_logits)

In [ ]:
answer = tokeniser.decode(input["input_ids"][0][start_idx:end_idx+1])
answer

'florence, italy'